In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 


from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from IPython.display import Image, HTML
import json
import datetime
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from wordcloud import WordCloud, STOPWORDS
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
sns.set(font_scale=1.25)
pd.set_option('display.max_colwidth', 50)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:

df1=pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv')
df2=pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv')

In [ ]:
df1.columns


In [ ]:
df1.columns = ['id','tittle','cast','crew']
df2= df2.merge(df1,on='id')

In [ ]:
df2.shape


Features
* budget: The budget of the movie in dollars.
* genres: A stringified list of dictionaries that list out all the genres associated with the movie.
* homepage: The Official Homepage of the move.
* id: The ID of the movie.
* imdb_id: The IMDB ID of the movie.
* original_language: The language in which the movie was originally shot in.
* original_title: The original title of the movie.
* overview: A brief blurb of the movie.
* popularity: The Popularity Score assigned by TMDB.
* poster_path: The URL of the poster image.
* production_companies: A stringified list of production companies involved with the making of the movie.¶
* production_countries: A stringified list of countries where the movie was shot/produced in.
* release_date: Theatrical Release Date of the movie.
* revenue: The total revenue of the movie in dollars.
* runtime: The runtime of the movie in minutes.
* spoken_languages: A stringified list of spoken languages in the film.
* status: The status of the movie (Released, To Be Released, Announced, etc.)
* tagline: The tagline of the movie.
* title: The Official Title of the movie.
* video: Indicates if there is a video present of the movie with TMDB.
* vote_average: The average rating of the movie.
* vote_count: The number of votes by users, as counted by TMDB.

In [ ]:
df2.columns


In [ ]:
df2.head()


In [ ]:
df2.info()


In [ ]:
df2[df2['original_title'] != df2['title']][['title', 'original_title']].head()


In [ ]:
df2 = df2.drop('original_title', axis=1)


In [ ]:
df2[df2['revenue'] == 0].shape


In [ ]:
df2['budget'] = pd.to_numeric(df2['budget'], errors='coerce')
df2['budget'] = df2['budget'].replace(0, np.nan)
df2[df2['budget'].isnull()].shape

In [ ]:
df2['return'] = df2['revenue'] / df2['budget']
df2[df2['return'].isnull()].shape

In [ ]:
df2['production_countries']


In [ ]:
df2['year'] = pd.to_datetime(df2['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [ ]:
df2['title'] = df2['title'].astype('str')
df2['overview'] = df2['overview'].astype('str')

In [ ]:
title_corpus = ' '.join(df2['title'])
overview_corpus = ' '.join(df2['overview'])

In [ ]:
# Create and generate a word cloud image:
wordcloud = WordCloud().generate(title_corpus)
plt.figure(figsize=(15,15))# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud().generate(overview_corpus)
plt.figure(figsize=(15,15))# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(df2.corr())
plt.show()

In [ ]:
from scipy.stats import pearsonr

sns.jointplot(x='budget', y='revenue', stat_func =pearsonr,data=df2)

we can definitely see some corelation between budget and revenue

In [ ]:
sns.jointplot(x='popularity', y='revenue', stat_func =pearsonr,data=df2)


again as expected , we see corelation between revenue and popularity

In [ ]:
sns.jointplot(x='vote_count', y='revenue', stat_func =pearsonr,data=df2)


strong corelation between revenue and vote_count . movies that have good returns are necessary to have a good critical score

In [ ]:
df2[['title', 'vote_count', 'year']].sort_values('vote_count', ascending=False).head(10)


In [ ]:
most_popular_movies_and_return = df2[['title', 'vote_count', 'year','revenue','return']].sort_values('vote_count', ascending=False)

In [ ]:
most_popular_movies_and_return=most_popular_movies_and_return[most_popular_movies_and_return['revenue']>=1000000000]

In [ ]:
# sns.catplot(x="Surface", data=injury,hue='BodyPart', kind="count")
chart = sns.catplot(x="year", data=most_popular_movies_and_return, kind="count")
chart.set_xticklabels(rotation=45)

plt.gcf().set_size_inches(24, 12)

after 1997 (titanic) which was the first movie to cross 1 billion dollar mark , 2012 and 2015 had many movies crossing the billion dollar mark

In [ ]:
df2[df2['vote_count'] > 3000][['title', 'vote_average', 'vote_count' ,'year']].sort_values('vote_average', ascending=False).head(10)

In [ ]:
sns.jointplot(x='vote_average', y='popularity', stat_func =pearsonr,data=df2)


In [ ]:
df2['genres'] = df2['genres'].fillna('[]').apply(ast.literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [ ]:
s = df2.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'

In [ ]:
gen_df = df2.drop('genres', axis=1).join(s)
df2.head()

In [ ]:
pop_gen = pd.DataFrame(gen_df['genre'].value_counts()).reset_index()
pop_gen.columns = ['genre', 'movies']
pop_gen.head(10)

In [ ]:
plt.figure(figsize=(18,8))
sns.barplot(x='genre', y='movies', data=pop_gen.head(15))
plt.show()

In [ ]:
res = df2.set_index(['popularity', 'revenue'])['genres'].apply(pd.Series).stack()
res = res.reset_index()
res.columns = ['popularity','revenue','sample_num','genre']
res.drop('sample_num',axis=1,inplace=True)
res

In [ ]:
res_400=res[res['revenue']>=400000000]


In [ ]:
df2[df2['budget'].notnull()][['title', 'budget', 'revenue', 'return', 'year']].sort_values('budget', ascending=False).head(10)

the johny depp starrer is the most expensive movie ever made !![](http://)

In [ ]:
gross_top = df2[['title', 'budget', 'revenue', 'year']].sort_values('revenue', ascending=False).head(10)
gross_top

the mighty avatar was the highest grossing movie of all time , surpassed recently by the avengers endgame . 

In [ ]:

plt.figure(figsize=(18,5))
year_revenue = df2[(df2['revenue'].notnull()) & (df2['year'] != 'NaT')].groupby('year')['revenue'].max()
# plt.plot(year_revenue.index, year_revenue)
# chart = plt.plot(year_revenue.index, year_revenue)
# chart.set_xticklabels(rotation=45)
# plt.gcf().set_size_inches(16, 8)
# plt.show()
year_revenue.plot(kind='line')

we can see that the revenue has been steadily increasing over the years 

In [ ]:
s = df2.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'

In [ ]:
gen_df = df2.drop('genres', axis=1).join(s)


In [ ]:
pop_genre = pd.DataFrame(gen_df['genre'].value_counts()).reset_index()
pop_genre.columns = ['genre', 'movies']
pop_gen.head(10)

In [ ]:
plt.figure(figsize=(18,8))
sns.barplot(x='genre', y='movies', data=pop_gen.head(15))
plt.show()

drama,comedy and thriller lead the way !!